## Loading NZDOC weights into PyTorch and compiling to Torchscript

Things to note: 
- the model was trained on a GPU so we need to load weights and re-compile to CPU
- it's important to check what version of torchvision (if used here) and torch you're running in this notebook environment & be sure they match the versions pinned in the deployment container's Dockerfile
- it's important to know which version of efficientnet was used in training (in nzdoc's case it was [EfficientNetV2](https://pytorch.org/vision/stable/models/efficientnetv2.html)) and the size of the inputs (480x480).

In [1]:
import torch
# from efficientnet_pytorch import EfficientNet

torch.__version__

'2.0.0'

In [2]:
# ckpt = torch.load("model-weights/ckpt_18.pt", map_location=torch.device("cpu"))
# model = EfficientNet.from_pretrained("efficientnet-b3", num_classes=5)
# model.load_state_dict(ckpt['model'])

model = torch.jit.load("model-weights/model_scripted.pt", map_location=torch.device("cpu"))

In [3]:
model.eval()

RecursiveScriptModule(
  original_name=EfficientNet
  (features): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=Conv2dNormActivation
      (0): RecursiveScriptModule(original_name=Conv2d)
      (1): RecursiveScriptModule(original_name=BatchNorm2d)
      (2): RecursiveScriptModule(original_name=SiLU)
    )
    (1): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=FusedMBConv
        (block): RecursiveScriptModule(
          original_name=Sequential
          (0): RecursiveScriptModule(
            original_name=Conv2dNormActivation
            (0): RecursiveScriptModule(original_name=Conv2d)
            (1): RecursiveScriptModule(original_name=BatchNorm2d)
            (2): RecursiveScriptModule(original_name=SiLU)
          )
        )
        (stochastic_depth): RecursiveScriptModule(original_name=StochasticDepth)
      )
      (1): RecursiveScriptModule(
        origi

In [4]:
# Save out the whole model for future inference deployment
# https://pytorch.org/tutorials/beginner/saving_loading_models.html

compiled_path = './model-weights/nzdoc_compiled_cpu.pt'

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(compiled_path) # Save